In [ ]:
import numpy as np
from collections import Counter
import pickle
import matplotlib.pyplot as plt
from decisiontree import DecisionTree

Coder une fonction entropie(vect) qui calcule l’entropie de ce vecteur

In [ ]:
def entropie(vect):
    c = Counter()
    c.update(vect)
    probas = np.array(list(c.values())) / len(vect)
    return -np.array([y * np.log(y) for y in probas]).sum()

def cond_entropie(list_vect):
    res = 0
    total = np.array([len(v) for v in list_vect]).sum()
    for vect in list_vect:
        res += (len(vect)/total) * entropie(vect)
    return res

In [ ]:
#entropie([1,2])
cond_entropie([[1,6,2],[9,1,2]])

In [ ]:
import pickle
import numpy as np
# data : tableau ( films , features ) , id2titles : dictionnaire id -> titre ,
# fields : id feature -> nom
[ data , id2titles , fields ]= pickle.load(open("imdb_extrait.pkl","rb"))
# la derniere colonne est le vote
datax = data [: ,:32]
datay = np.array([1 if x [33] >6.5 else -1 for x in data ])

binary_columns = data[:,:27]
N = len(data)
print(N)

In [ ]:
binary_columns

In [ ]:
dt = DecisionTree()
dt.max_depth = 10
dt.min_samples_split = 2 # nombre minimum d ’ exemples pour spliter un noeud
dt.fit(datax , datay )
dt.predict( datax [:5 ,:])
print(dt.score( datax , datay ))
# dessine l ’ arbre dans un fichier pdf si pydot est installe .
#dt.to_pdf("/tmp/test_tree.pdf ",fields)
# sinon utiliser http :// www . webgraphviz . com /
dt.to_dot(fields)
# ou dans la console
print(dt.print_tree(fields))

In [ ]:
from decisiontree import DecisionTree
for partition in [(0.2,0.8, 'r'),(0.5,0.5, 'b'),(0.8,0.2, 'g')]:
    print(partition)
    erreur_train = []
    erreur_test = []
    training_x, test_x, training_y, test_y = train_test_data(datax, datay, partition[0])
    for depth in range(1,10):
        dt = DecisionTree()
        dt.max_depth = depth
        dt.min_samples_split = 2 # nombre minimum d ’ exemples pour spliter un noeud
        dt.fit(training_x , training_y )
        erreur_train.append(dt.score(training_x, training_y))
        erreur_test.append(dt.score(test_x,test_y))
    plt.plot(list(range(len(erreur_train))), erreur_train, partition[2])
    plt.plot(list(range(len(erreur_test))), erreur_test, partition[2]+'-')
plt.show()

Le partition 0.8 / 0.2 (vert) semble d'etre le plus robust pour test et training.

Quand il y a que peu d'examples d'apprentissage, l'erreur est plus éleve dans le test. Par contre quand on a beacoup des d'examples d'aprentissage, l'erreur dans le test est moins d'élevé.

In [ ]:
def train_test_data(x,y, partition):
    train_n = int(len(x) * partition)
    test_n = int(len(x) - train_n)
    indices = np.random.permutation(len(x))
    training_idx, test_idx = indices[:train_n], indices[test_n:]
    return(x[training_idx,:], x[test_idx,:] ,y[training_idx], y[test_idx])

In [ ]:

dt = DecisionTree()
dt.max_depth = 4
dt.min_samples_split = 2 # nombre minimum d ’ exemples pour spliter un noeud
for partition in [(0.2,0.8)]:#[(0.2,0.8),(0.5,0.5),(0.8,0.2)]:
    training_x, test_x, training_y, test_y = train_test_data(datax, datay, partition[0])
    dt.fit(training_x , training_y )
    print(type(training_x[0]))
    print(type(training_y))
    #result = dt.predict(test_x)
    #print(dt.score( np.array(result) , test_y ))
    # dessine l ’ arbre dans un fichier pdf si pydot est installe .
    #dt.to_pdf("/tmp/test_tree.pdf ",fields)
    # sinon utiliser http :// www . webgraphviz . com /
    dt.to_dot(fields)
    # ou dans la console
    #print(dt.print_tree(fields))

In [ ]:
def divide_intervalles(X, Y, N):
    assert X.shape[0] == Y.shape[0]
    indexes = np.arange(X.shape[0])
    np.random.shuffle(indexes)
    index_intevals = np.array_split(indexes, N)
    res = np.array([(X[ind, :],Y[ind]) for ind in index_intevals])
    return (res[:, 0], res[:, 1])

def validation_croisee(X,Y, dt, N):
    intervalles_x, intervalles_y = divide_intervalles(X,Y,N)
    results = []
    for i in range(N):
        train_x = []
        train_y = []
        test_x = []
        test_y = []
        for j in range(N):
            if i != j:
                train_x.extend(intervalles_x[j])
                train_y.extend(intervalles_y[j])
            else:
                test_x.extend(intervalles_x[j])
                test_y.extend(intervalles_y[j])
        dt.fit(np.array(train_x), np.array(train_y))
        results.append(dt.score(np.array(test_x), np.array(test_y)))
    return np.array(results).mean()

188 / 10

In [ ]:
[1,2] + np.array([3])

In [ ]:
dt = DecisionTree()
dt.max_depth = 4
dt.min_samples_split = 2 # nombre minimum d ’ exemples pour spliter un noeud
print(validation_croisee(datax, datay, dt, 5))

In [ ]:
4587 / 10